In [1]:
import torch
import tiktoken

In [2]:
with open ("one-foggy-night.txt", "r" ) as f:
    raw_text = f.read()

print (raw_text[:50])

A PLUME of smoke drifted under the great glass dom


In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
enc_text = tokenizer.encode(raw_text)

In [5]:
print(enc_text[:20])

[32, 9297, 38340, 286, 7523, 38648, 739, 262, 1049, 5405, 29500, 286, 3454, 363, 17913, 9327, 355, 262, 8342, 10604]


In [6]:
print( tokenizer.decode( enc_text[:2]))
len( enc_text)

A PL


5513

In [7]:
for i in range(1,10):
    print("Input:", tokenizer.decode(enc_text[:i]), "Target:", tokenizer.decode([enc_text[i]]))

Input: A Target:  PL
Input: A PL Target: UME
Input: A PLUME Target:  of
Input: A PLUME of Target:  smoke
Input: A PLUME of smoke Target:  drifted
Input: A PLUME of smoke drifted Target:  under
Input: A PLUME of smoke drifted under Target:  the
Input: A PLUME of smoke drifted under the Target:  great
Input: A PLUME of smoke drifted under the great Target:  glass


In [8]:
print("PyTorch version:", torch.__version__)

PyTorch version: 2.2.2


In [9]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [11]:
with open("one-foggy-night.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [12]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   32,  9297, 38340,   286]]), tensor([[ 9297, 38340,   286,  7523]])]


In [13]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 9297, 38340,   286,  7523]]), tensor([[38340,   286,  7523, 38648]])]


In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   32,  9297, 38340,   286],
        [ 7523, 38648,   739,   262],
        [ 1049,  5405, 29500,   286],
        [ 3454,   363, 17913,  9327],
        [  355,   262,  8342, 10604],
        [ 1625,  6364,   284,   257],
        [ 1302, 24219,    13,   632],
        [  373,  1972,  2739,   783]])

Targets:
 tensor([[ 9297, 38340,   286,  7523],
        [38648,   739,   262,  1049],
        [ 5405, 29500,   286,  3454],
        [  363, 17913,  9327,   355],
        [  262,  8342, 10604,  1625],
        [ 6364,   284,   257,  1302],
        [24219,    13,   632,   373],
        [ 1972,  2739,   783,    11]])


In [16]:

with open("one-foggy-night.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[   32,  9297, 38340,   286]]), tensor([[ 9297, 38340,   286,  7523]])]
[tensor([[ 9297, 38340,   286,  7523]]), tensor([[38340,   286,  7523, 38648]])]
